### Versiones de las librerías utilizadas

- sklearn <u>***version 1.0.2***</u>
- bs4 <u>***version 4.11.1***</u>
- requests <u>***version 2.27.1***</u>
- pandas <u>***version 1.4.2***</u>
- wordfreq <u>***version 3.1.1***</u>
- numpy <u>***version 1.22.4***</u>
- spacy <u>***version 3.7.5***</u>

### Importe de librerias y métodos necesarios

In [189]:
# librerias de scraping
import requests
from bs4 import BeautifulSoup

# librerias de procesamiento de data
import pandas as pd
import numpy as np

# librerias de procesamiento de texto
import spacy
from wordfreq import zipf_frequency

# librerias para la construcción de los modelos
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import chi2_kernel
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV  
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix

In [ ]:
#nlpes = spacy.load('es_core_news_sm')
#nlpen = spacy.load('en_core_web_sm')
#spanishWords['word'] = [nlpes(x)[0].lemma_ for x in spanishWords[0]]
#englishhWords['word'] = [nlpen(x)[0].lemma_ for x in englishhWords[0]]

#bagWordsBalancedExploded = data.explode('freq').reset_index()
#documents = []
#for language in bagWordsBalancedExploded['language'].unique():
#    language = bagWordsBalancedExploded[bagWordsBalancedExploded['language']==language]
#    documents.append(' '.join(language['word']))

#import pyximport; pyximport.install()  # noqa: E702
#from string_kernel import ssk, string_kernel

## Funciones de utilidad

In [45]:
def freedman_diaconis(data):
    """
    Utiliza el método de Freedman para el cálculo de particiones en un histograma
    
    Args:
        data: Columna con los valores a particionar
        
    Returns:
        el número de bins (particiones) ideales
    """
    iqr = np.percentile(data, 75) - np.percentile(data, 25)
    h = 2 * iqr * len(data)**(-1/3)
    bins = np.arange(min(data), max(data) + h, h)
    return bins

In [15]:
def bigrams(text):
    """
    Genera una lista de bigramas a partir de un texto.

    Args:
        text: El texto a procesar.

    Returns:
        Una lista de bigramas.
    """
    # Toma una palabra y lo segmenta en secuencia de letras tan grande como la n plasmada
    bigramas = []
    for i in range(len(text) - 1):
        bigramas.append(text[i:i+2])
    return bigramas

## Extracción y procesamiento de la data cruda

In [9]:
base_url = 'https://en.wiktionary.org'
spanish = requests.get('https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/Spanish/Mixed_730K')
english = requests.get('https://en.wiktionary.org/wiki/Wiktionary:Frequency_lists/English/Wikipedia_(2016)')

def buildWordsByLanguage(response,language):
    """
    Construye la base de palabra, idioma desde la página brindada.

    Args:
        response: respuesta del request a la página.
        language: idioma a buscar en la página.

    Returns:
        Un dataframe con la palabra y su idioma. 
    """

    if response.status_code == 200:
        
        # Conexión a la página con la data
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extracción de la lista de palabras de la página principal
        languageWords = soup.find_all('p')
        languageWords = [x.find_all('a', href=True) if hasattr(x, 'find_all') else [] for x in languageWords]
        languageWords = [y.text for x in [x for x in languageWords if f'#{language}' in str(x)] for y in x]
        
        # Establecer el límite de palabrar para la búsqueda
        limitLanguageWords = soup.find_all('li')
        limitLanguageWords = [x.text for x in limitLanguageWords if 'Wiktionary:Frequency_lists' in str(x) and '0"' in str(x)]
        
        # Lista de las subpáginas con el resto de palabras
        links = soup.find_all('a', href=True)
        links = [[x for x in links if y in str(x)] for y in limitLanguageWords]
        links = [f"{base_url}{y.get_attribute_list('href')[0]}" for x in links for y in x]
        
        # Extracción de la lista de palabras de las subpáginas
        for linkWords in links:
            languageAux = requests.get(linkWords)
            soupAux = BeautifulSoup(languageAux.content, 'html.parser')
            languageAux = soupAux.find_all('p')
            languageAux = [x.find_all('a', href=True) if hasattr(x, 'find_all') else [] for x in languageAux]
            languageAux = [y.text for x in [x for x in languageAux if '#Spanish' in str(x)] for y in x]
            languageWords = languageWords+languageAux
        
        # Construcción de la lista final de palabras y su etiqueta correspondiente
        languageWords = [x.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u') for x in languageWords if len(x)>=4]
        languageWords = dict(zip(languageWords,[language]*len(languageWords)))
        languageWords = pd.DataFrame.from_dict(data=[languageWords.keys(),languageWords.values()]).T
        
        # Generación de la frecuencia mediante un corpus predeterminado y filtrado por frecuencia de las palabras
        languageWords['freq'] = [zipf_frequency(x, 'es') if language == 'Spanish' else zipf_frequency(x, 'en') if language == 'English' else 0 for x in languageWords[0]]
        languageWords = languageWords[languageWords['freq']>1]
    else:
        print("Request failed with status code:", response.status_code)

    return languageWords

In [ ]:
spanishWords = buildWordsByLanguage(spanish,'Spanish')
englishWords = buildWordsByLanguage(english,'English')

In [18]:
spanishWords.head(5)

,0,1,freq
0,para,Spanish,6.92
1,como,Spanish,6.73
2,este,Spanish,6.37
3,pero,Spanish,6.57
4,años,Spanish,6.12


## Construcción del dataset para los modelos

In [29]:
def dataSetforModel(listWordsLanguage,sampleSize):
    """
    Genera un dataframe con igual cantidad de datos por etiqueta para el balanceo.

    Args:
        listWordsLanguage: lista generada con cada dataframe palabra-idioma.
        sampleSize: numero de muestras por dataframe de idioma.

    Returns:
        Un dataframe balanceado por etiquetas.
    """
        
    # Creación de lista de dataframes de palabras-idiomas sampelados para balanceo de data
    bagWords = []
    for language in listWordsLanguage:
        bagWords.append(language.sample(n=sampleSize, replace=False, random_state=42))
    
    # Concatenación de las muestras por lenguaje y homologación del formato de las palabras.
    # Construcción de bigramas en las palabras y reformulación de la frecuencia para un posible uso con explode    
    bagWords = pd.concat(bagWords).reset_index().drop('index',axis=1).rename(columns={0:'word',1:'language'})
    bagWords['freq'] = np.ceil(bagWords['freq'])
    bagWords['freq'] = [list(range(1,int(x))) for x in bagWords['freq']]
    bagWords['word'] = [x.lower() for x in bagWords['word']]
    bagWords['bigrams'] = bagWords['word'].apply(bigrams)
    
    return bagWords

In [30]:
bagWordsBalanced = dataSetforModel([spanishWords,englishWords],1000)

In [31]:
bagWordsBalanced.head(5)

,word,language,freq,bigrams
0,chemical,Spanish,"[1, 2]","[ch, he, em, mi, ic, ca, al]"
1,diario de avisos,Spanish,"[1, 2, 3]","[di, ia, ar, ri, io, o , d, de, e , a, av, v..."
2,alberto martin,Spanish,"[1, 2, 3, 4]","[al, lb, be, er, rt, to, o , m, ma, ar, rt, t..."
3,afirmamos,Spanish,"[1, 2]","[af, fi, ir, rm, ma, am, mo, os]"
4,h-k-p-t,Spanish,"[1, 2, 3, 4]","[h-, -k, k-, -p, p-, -t]"


## Coisine Kernel

In [32]:
def coisineKernel(data,testSize):
    """
    Calcula la similaridad del Coseno para las palabras contrastando palabra vs palabra.

    Args:
        data: Dataframe con la data para el modelo.
        testSize: porcentaje del conjunto de prueba respecto al total de la data.

    Returns:
        La matriz de similariad entre palabras.
    """

    # Separar las características (palabras) y las etiquetas (idioma)
    X = data['word']
    y = data['language']

    # División del dataset en entrenamiento y prueba
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize)

    # Crear un vectorizador de conteo
    vectorizer = CountVectorizer(ngram_range=(1,3))

    # Transformar los datos en una matriz de conteo
    X_train_X = vectorizer.fit_transform(X_train)
    
    # Calcular la similitud del coseno
    cosine_sim = cosine_similarity(X_train_X)
    
    return cosine_sim

In [33]:
coisineKernel(bagWordsBalanced,0.3)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

## Coisine Kernel V2

In [54]:
def coisineKernel2(data):
    """
    Calcula la similaridad del Coseno para las palabras contrastando ´Documentos x idioma´.

    Args:
        data: Dataframe con la data para el modelo.
        testSize: porcentaje del conjunto de prueba respecto al total de la data.

    Returns:
        La matriz de similariad entre documentos.
    """
    # Creación de "documentos" por unión de palabras 
    documents = []
    for language in data['language'].unique():
        language = data[data['language']==language]
        documents.append(' '.join(language['word']))

    # Crear un vectorizador de conteo
    vectorizer = CountVectorizer(ngram_range=(1, 3))

    # Transformar los documentos en una matriz de conteo
    X = vectorizer.fit_transform(documents)

    # Calcular la similitud del coseno
    cosine_sim = cosine_similarity(X)

    return cosine_sim

In [56]:
coisineKernel2(bagWordsBalanced)

array([[1.        , 0.04218385],
       [0.04218385, 1.        ]])

## Suma de los mínimos de los intervalos

In [57]:
def sumMinInt(data):
    """
    Calcula los intervalos de numero de palabras de cada idioma y suma los minimos de cada intervalo
    
    Args:
        data: Dataframe de palabras-idioma
    
    Returns:
        La suma de los minimos de cada intervalo para cada "idioma"
    """

    # Creación de "documentos" por unión de palabras 
    documents = []
    for language in data['language'].unique():
        language = data[data['language']==language]
        documents.append(' '.join(language['word']))
    
    # Crear un vectorizador de conteo
    vectorizer = CountVectorizer(ngram_range=(1, 3))
    
    # Convertir la matriz a un array de NumPy para facilitar el procesamiento
    X_train_X = vectorizer.fit_transform(documents)
    X_train_X = X_train_X.toarray()

    # Normalizar cada fila (documento)
    normalized_X = X_train_X / np.sum(X_train_X, axis=1, keepdims=True)

    # Crea un histograma y calcular la suma del mínimo por bin
    mins = []
    for i in range(normalized_X.shape[0]):
        aux = sorted(normalized_X[i])
        bins = freedman_diaconis(aux)
        pos = [np.where((aux>=bins[x-1]) & (aux<bins[x]))[0] for x in range(1,len(bins))]
        pos = [min(x) for x in pos if len(x)!=0]
        sum_of_min  = sum([aux[x] for x in range(len(aux)) if x in pos])
        mins.append(f"Idioma {i+1}: Suma del mínimo por bin = {sum_of_min}")
    return mins

In [59]:
sumMinInt(bagWordsBalanced)

['Idioma 1: Suma del mínimo por bin = 0.026505513146734522',
 'Idioma 2: Suma del mínimo por bin = 0.021945432977461446']

## kernel Chi-cuadrado

Por el mismo formato de la data la matriz resultante para aplicar el kernel es dispersa y el kernel chi cuadrado por su formulación tiene problemas para tratar con matrices con muchos ceros.

In [60]:
def chi2Kernel(data):
    """
    
    """
    # Creación de "documentos" por unión de palabras 
    documents = []
    for language in data['language'].unique():
        language = data[data['language']==language]
        documents.append(' '.join(language['word']))
    
    # Crear un vectorizador de conteo
    vectorizer = CountVectorizer(ngram_range=(1, 3))
    
    X = vectorizer.fit_transform(documents)

    # Calcular el kernel chi-cuadrado
    kernel_matrix = chi2_kernel(X)
    
    return kernel_matrix

In [61]:
chi2Kernel(bagWordsBalanced)

ValueError: additive_chi2 does not support sparse matrices.

## SSK Kernel

**Link correcto:** https://github.com/helq/python-ssk/blob/master/main.py

No se pudo usar porque se necesitaba instalación de versiones de C no soportadas. Intenté instalarlo en mi pc pero no fue posible. Intenté usar el código directamente pero necesitaba adaptarlo, y ya que no está muy bien documentado y considero que está fuera de alcance de la prueba no lo realicé.

Por favor asegurarse de proporcionar todo el software que las personas necesitan para poder realizar las tareas.

**Error:**

Building module string_kernel failed: ['distutils.errors.DistutilsPlatformError: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/\n']

## Entrenamiento SVMs

In [186]:
def SVMsModels(data,testSize,numGramas):
    """
    Genera diferentes combinaciones de parámetros para un modelo SVM y entrega el mejor estimador.
    
    Args:
        data: Dataframe con los datospara el modelo.
        testSize: porcentaje del conjunto de prueba respecto al total de la data.
        numGramas: número de gramas para el countVectorizer
        
    Returns:
        la malla de combinaciones de hyperparametros.
    """
    # Separar las características (palabras) y las etiquetas (idioma)
    #X = bagWordsBalanced[['word','bigrams']]
    X = data['word']
    y = data['language']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testSize)

    vectorizerTest = CountVectorizer(ngram_range=(numGramas, numGramas))
    X_test_X = vectorizerTest.fit_transform(X_test)
    
    vectorizerTrain = CountVectorizer(ngram_range=(numGramas, numGramas),max_features=X_test_X.shape[1])  # Unigramas, bigramas y trigramas, cuatrigramas, quintigramas
    #X = vectorizer.fit_transform(X_train)

    # Transformar los datos
    X_train_X = vectorizerTrain.fit_transform(X_train)

    # Definir los parámetros a ajustar
    param_grid = {'C': [2**x for x in range(-15,27)],
                  'kernel': ['linear', 'rbf','poly'],
                  'gamma': [0.001, 0.01, 0.1]}

    # Crear el modelo SVM y utilizar GridSearchCV
    grid = GridSearchCV(SVC(), param_grid, refit=True, verbose=3)
    grid.fit(X_train_X, y_train)
    
    return grid,X_test_X,y_test,X_train_X,y_train

In [70]:
#SVMsModels(bagWordsBalanced,0.3,4)

## Resultados SVMs

In [180]:
malla = []
x_test = []
y_test = []
x_train = []
y_train = []
for gramas in [1,2,3,4,5,6]:
    model = SVMsModels(bagWordsBalanced,0.3,gramas)
    malla.append(model[0])
    x_test.append(model[1])
    y_test.append(model[2])
    x_train.append(model[3])
    y_train.append(model[4])

Fitting 5 folds for each of 378 candidates, totalling 1890 fits
[CV 1/5] END C=3.0517578125e-05, gamma=0.001, kernel=linear;, score=0.507 total time=   0.0s
[CV 2/5] END C=3.0517578125e-05, gamma=0.001, kernel=linear;, score=0.504 total time=   0.0s
[CV 3/5] END C=3.0517578125e-05, gamma=0.001, kernel=linear;, score=0.504 total time=   0.0s
[CV 4/5] END C=3.0517578125e-05, gamma=0.001, kernel=linear;, score=0.504 total time=   0.0s
[CV 5/5] END C=3.0517578125e-05, gamma=0.001, kernel=linear;, score=0.504 total time=   0.0s
[CV 1/5] END C=3.0517578125e-05, gamma=0.001, kernel=rbf;, score=0.507 total time=   0.0s
[CV 2/5] END C=3.0517578125e-05, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 3/5] END C=3.0517578125e-05, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 4/5] END C=3.0517578125e-05, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 5/5] END C=3.0517578125e-05, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 1/5] END C=3.0517

[CV 2/5] END C=0.0001220703125, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.0001220703125, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.0001220703125, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.0001220703125, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.0001220703125, gamma=0.001, kernel=poly;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.0001220703125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.0001220703125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.0001220703125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.0001220703125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.0001220703125, gamma=0.01, kernel=linear;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.0001220703125, gamma=0.01, kernel=linear;, score=0.504 total time=   0.0s
[CV 3/5] END

[CV 5/5] END C=0.00048828125, gamma=0.001, kernel=rbf;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.00048828125, gamma=0.001, kernel=poly;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.00048828125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.00048828125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.00048828125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.00048828125, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.00048828125, gamma=0.01, kernel=linear;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.00048828125, gamma=0.01, kernel=linear;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.00048828125, gamma=0.01, kernel=linear;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.00048828125, gamma=0.01, kernel=linear;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.00048828125, gamma=0.01, kernel=linear;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.00048828125

[CV 1/5] END C=0.001953125, gamma=0.01, kernel=rbf;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.001953125, gamma=0.01, kernel=rbf;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.001953125, gamma=0.01, kernel=rbf;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.001953125, gamma=0.01, kernel=rbf;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.001953125, gamma=0.01, kernel=rbf;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.001953125, gamma=0.01, kernel=poly;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.001953125, gamma=0.01, kernel=poly;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.001953125, gamma=0.01, kernel=poly;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.001953125, gamma=0.01, kernel=poly;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.001953125, gamma=0.01, kernel=poly;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.001953125, gamma=0.1, kernel=linear;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.001953125, gamma=0.1, kernel=linear;, score=0.504 to

[CV 3/5] END C=0.0078125, gamma=0.1, kernel=linear;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.0078125, gamma=0.1, kernel=linear;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.0078125, gamma=0.1, kernel=linear;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.0078125, gamma=0.1, kernel=rbf;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.0078125, gamma=0.1, kernel=rbf;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.0078125, gamma=0.1, kernel=rbf;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.0078125, gamma=0.1, kernel=rbf;, score=0.504 total time=   0.0s
[CV 5/5] END C=0.0078125, gamma=0.1, kernel=rbf;, score=0.504 total time=   0.0s
[CV 1/5] END C=0.0078125, gamma=0.1, kernel=poly;, score=0.507 total time=   0.0s
[CV 2/5] END C=0.0078125, gamma=0.1, kernel=poly;, score=0.504 total time=   0.0s
[CV 3/5] END C=0.0078125, gamma=0.1, kernel=poly;, score=0.504 total time=   0.0s
[CV 4/5] END C=0.0078125, gamma=0.1, kernel=poly;, score=0.504 total time=   0.0s
[CV 5/5] END C=

...

[CV 3/5] END C=131072, gamma=0.001, kernel=linear;, score=0.550 total time=   0.0s
[CV 4/5] END C=131072, gamma=0.001, kernel=linear;, score=0.543 total time=   0.0s
[CV 5/5] END C=131072, gamma=0.001, kernel=linear;, score=0.582 total time=   0.0s
[CV 1/5] END .C=131072, gamma=0.001, kernel=rbf;, score=0.546 total time=   0.0s
[CV 2/5] END .C=131072, gamma=0.001, kernel=rbf;, score=0.564 total time=   0.0s
[CV 3/5] END .C=131072, gamma=0.001, kernel=rbf;, score=0.550 total time=   0.0s
[CV 4/5] END .C=131072, gamma=0.001, kernel=rbf;, score=0.543 total time=   0.0s
[CV 5/5] END .C=131072, gamma=0.001, kernel=rbf;, score=0.586 total time=   0.0s
[CV 1/5] END C=131072, gamma=0.001, kernel=poly;, score=0.504 total time=   0.0s
[CV 2/5] END C=131072, gamma=0.001, kernel=poly;, score=0.507 total time=   0.0s
[CV 3/5] END C=131072, gamma=0.001, kernel=poly;, score=0.507 total time=   0.0s
[CV 4/5] END C=131072, gamma=0.001, kernel=poly;, score=0.507 total time=   0.0s
[CV 5/5] END C=131072,

[CV 2/5] END C=524288, gamma=0.01, kernel=linear;, score=0.561 total time=   0.0s
[CV 3/5] END C=524288, gamma=0.01, kernel=linear;, score=0.550 total time=   0.0s
[CV 4/5] END C=524288, gamma=0.01, kernel=linear;, score=0.543 total time=   0.0s
[CV 5/5] END C=524288, gamma=0.01, kernel=linear;, score=0.582 total time=   0.0s
[CV 1/5] END ..C=524288, gamma=0.01, kernel=rbf;, score=0.664 total time=   0.0s
[CV 2/5] END ..C=524288, gamma=0.01, kernel=rbf;, score=0.689 total time=   0.0s
[CV 3/5] END ..C=524288, gamma=0.01, kernel=rbf;, score=0.629 total time=   0.0s
[CV 4/5] END ..C=524288, gamma=0.01, kernel=rbf;, score=0.661 total time=   0.0s
[CV 5/5] END ..C=524288, gamma=0.01, kernel=rbf;, score=0.643 total time=   0.0s
[CV 1/5] END .C=524288, gamma=0.01, kernel=poly;, score=0.518 total time=   0.0s
[CV 2/5] END .C=524288, gamma=0.01, kernel=poly;, score=0.521 total time=   0.0s
[CV 3/5] END .C=524288, gamma=0.01, kernel=poly;, score=0.518 total time=   0.0s
[CV 4/5] END .C=524288, 

[CV 5/5] END C=2097152, gamma=0.01, kernel=poly;, score=0.543 total time=   0.0s
[CV 1/5] END C=2097152, gamma=0.1, kernel=linear;, score=0.546 total time=   0.0s
[CV 2/5] END C=2097152, gamma=0.1, kernel=linear;, score=0.561 total time=   0.0s
[CV 3/5] END C=2097152, gamma=0.1, kernel=linear;, score=0.550 total time=   0.0s
[CV 4/5] END C=2097152, gamma=0.1, kernel=linear;, score=0.543 total time=   0.0s
[CV 5/5] END C=2097152, gamma=0.1, kernel=linear;, score=0.582 total time=   0.0s
[CV 1/5] END ..C=2097152, gamma=0.1, kernel=rbf;, score=0.661 total time=   0.0s
[CV 2/5] END ..C=2097152, gamma=0.1, kernel=rbf;, score=0.689 total time=   0.0s
[CV 3/5] END ..C=2097152, gamma=0.1, kernel=rbf;, score=0.629 total time=   0.0s
[CV 4/5] END ..C=2097152, gamma=0.1, kernel=rbf;, score=0.664 total time=   0.0s
[CV 5/5] END ..C=2097152, gamma=0.1, kernel=rbf;, score=0.639 total time=   0.0s
[CV 1/5] END .C=2097152, gamma=0.1, kernel=poly;, score=0.536 total time=   0.0s
[CV 2/5] END .C=2097152

[CV 4/5] END .C=8388608, gamma=0.1, kernel=poly;, score=0.543 total time=   0.0s
[CV 5/5] END .C=8388608, gamma=0.1, kernel=poly;, score=0.543 total time=   0.0s
[CV 1/5] END C=16777216, gamma=0.001, kernel=linear;, score=0.546 total time=   0.0s
[CV 2/5] END C=16777216, gamma=0.001, kernel=linear;, score=0.561 total time=   0.0s
[CV 3/5] END C=16777216, gamma=0.001, kernel=linear;, score=0.550 total time=   0.0s
[CV 4/5] END C=16777216, gamma=0.001, kernel=linear;, score=0.543 total time=   0.0s
[CV 5/5] END C=16777216, gamma=0.001, kernel=linear;, score=0.582 total time=   0.0s
[CV 1/5] END C=16777216, gamma=0.001, kernel=rbf;, score=0.546 total time=   0.0s
[CV 2/5] END C=16777216, gamma=0.001, kernel=rbf;, score=0.564 total time=   0.0s
[CV 3/5] END C=16777216, gamma=0.001, kernel=rbf;, score=0.550 total time=   0.0s
[CV 4/5] END C=16777216, gamma=0.001, kernel=rbf;, score=0.543 total time=   0.0s
[CV 5/5] END C=16777216, gamma=0.001, kernel=rbf;, score=0.586 total time=   0.0s
[CV

[CV 4/5] END C=67108864, gamma=0.001, kernel=rbf;, score=0.543 total time=   0.0s
[CV 5/5] END C=67108864, gamma=0.001, kernel=rbf;, score=0.586 total time=   0.0s
[CV 1/5] END C=67108864, gamma=0.001, kernel=poly;, score=0.518 total time=   0.0s
[CV 2/5] END C=67108864, gamma=0.001, kernel=poly;, score=0.521 total time=   0.0s
[CV 3/5] END C=67108864, gamma=0.001, kernel=poly;, score=0.518 total time=   0.0s
[CV 4/5] END C=67108864, gamma=0.001, kernel=poly;, score=0.511 total time=   0.0s
[CV 5/5] END C=67108864, gamma=0.001, kernel=poly;, score=0.532 total time=   0.0s
[CV 1/5] END C=67108864, gamma=0.01, kernel=linear;, score=0.546 total time=   0.0s
[CV 2/5] END C=67108864, gamma=0.01, kernel=linear;, score=0.561 total time=   0.0s
[CV 3/5] END C=67108864, gamma=0.01, kernel=linear;, score=0.550 total time=   0.0s
[CV 4/5] END C=67108864, gamma=0.01, kernel=linear;, score=0.543 total time=   0.0s
[CV 5/5] END C=67108864, gamma=0.01, kernel=linear;, score=0.582 total time=   0.0s
[

In [181]:
gramTable = []
for best in malla:
    gramTable.append([malla.index(best)+1,best.best_score_,best.best_params_,best.best_estimator_])

In [182]:
pd.DataFrame(gramTable,columns=['# gramas','accuracy','params','estimator'])

,# gramas,accuracy,params,estimator
0,1,0.658571,"{'C': 4, 'gamma': 0.1, 'kernel': 'rbf'}","SVC(C=4, gamma=0.1)"
1,2,0.661429,"{'C': 2, 'gamma': 0.1, 'kernel': 'rbf'}","SVC(C=2, gamma=0.1)"
2,3,0.650714,"{'C': 2, 'gamma': 0.1, 'kernel': 'rbf'}","SVC(C=2, gamma=0.1)"
3,4,0.663571,"{'C': 4, 'gamma': 0.1, 'kernel': 'rbf'}","SVC(C=4, gamma=0.1)"
4,5,0.656429,"{'C': 4, 'gamma': 0.1, 'kernel': 'rbf'}","SVC(C=4, gamma=0.1)"
5,6,0.661429,"{'C': 4, 'gamma': 0.1, 'kernel': 'rbf'}","SVC(C=4, gamma=0.1)"


In [188]:
y_pred = gramTable[1][3].predict(x_test[1])

## Discusión de resultados

Aunque en casos reales una predicción del 66% resulta relativamente bien, utilizar solamente la palabra como variable para predecir carece de mucho sentido dado que se pierden las propiedades sintácticas que enriquecen el idioma y que permitirian un mejor estimador.

Una matriz de confusión representa los valores de predicción anteriores.

La interpretabilidad al momento de generar la data para los análisis resulta en un sesgo de interpretación que afecta la calidad de los ejercicios planteados.

In [ ]:
# Calcular la matriz de confusión
cm = confusion_matrix(y_train[1], y_pred)